Skolemization-Related Explorations
==================

This is a temporary notebook for exploring issues related to a skolemize() method for KnownTruth objects, in particular for known truths involving universal ($\forall$) and existential ($\exists$) quantifiers.

The notebook was generated initially as a copy of the Forall tutorial notebook, and thus while under construction many of the cells and other elements of the original notebook remain.

($\forall$) Universal quantification is another core concept in **Prove-It**.  A `Forall` operation, formatted with the $\forall$ symbol, is used to represent universal quantification.  For example, $\forall_x P(x)$ means that $P(x)$ is true for any instance of $x$.  $P(x)$ holds true universally over instances of $x$.  Like `Implies`, `Forall` is a core concept but is defined outside of the core in the `proveit.logic` package. It is known in the core for use in the *specialization* and *generalization* derivation steps discussed below.

First, let us import some necessary information then consider an example of a `Forall` object like $\forall_{x \,\in\, S \,|\, Q(x), R(x)} P(x)$:

In [1]:
from proveit import Function, ExprList, Lambda, Literal
from proveit.logic import Forall, Exists, InSet, Equals
from proveit.logic.equality._axioms_ import substitution
from proveit.number import Less, Add, Frac, zero
from proveit._common_ import a, b, c, f, x, y, z, fx, P, Px, Pxy, Q, Qx, R, Rx, Ry, S, t, T
%begin skolemization_explorations

generalExpr =  forall_{A | [not](A)} (A != TRUE)
[_specialized_expression]subbed_expr =  FALSE != TRUE
[_specialized_expression]subbedConditions =  [[not](FALSE)]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), []]
[_specialized_expression]mappings =  {A: FALSE}
AFTER _specialized_expr:
    specializedExpr =  FALSE != TRUE
    requirements =  [[not](FALSE)]
    mappedVarLists =  [(A,), []]
    mappings =  {A: FALSE}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  FALSE != TRUE
generalTruth =  |- forall_{A | [not](A)} (A != TRUE)
requirementTruths =  [|- [not](FALSE)]
generalExpr =  forall_{x, y | x != y} (y != x)
[_specialized_expression]subbed_expr =  TRUE != FALSE
[_specialized_expression]subbedConditions =  [FALSE != TRUE]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: FALSE, y: TRU

[_specialized_expression]mappings =  {x: x, y: _x_}
AFTER _specialized_expr:
    specializedExpr =  (x = _x_) in BOOLEANS
    requirements =  []
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: x, y: _x_}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (x = _x_) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  (x = _x_) in BOOLEANS
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: x, y: _x_}
AFTER _specialized_expr:
    specializedExpr =  (x = _x_) in BOOLEANS
    requirements =  []
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: x, y: _x_}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTr

generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  (A = FALSE) in BOOLEANS
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: A, y: FALSE}
AFTER _specialized_expr:
    specializedExpr =  (A = FALSE) in BOOLEANS
    requirements =  []
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: A, y: FALSE}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (A = FALSE) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  (_x_ = FALSE) in BOOLEANS
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mapping

assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (_x_ = TRUE) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
generalExpr =  forall_{A | [not](A)} (A = FALSE)
generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  (TRUE = FALSE) in BOOLEANS
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: TRUE, y: FALSE}
AFTER _specialized_expr:
    specializedExpr =  (TRUE = FALSE) in BOOLEANS
    requirements =  []
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: TRUE, y: FALSE}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (TRUE = FALSE) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
[_specialized_expression]

AFTER _specialized_expr:
    specializedExpr =  [not](TRUE)
    requirements =  [TRUE = FALSE]
    mappedVarLists =  [(A,), []]
    mappings =  {A: TRUE}
generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  (TRUE = TRUE) in BOOLEANS
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: TRUE, y: TRUE}
AFTER _specialized_expr:
    specializedExpr =  (TRUE = TRUE) in BOOLEANS
    requirements =  []
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: TRUE, y: TRUE}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (TRUE = TRUE) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  (TRUE = TRUE) in BOOLEANS
[_specialized_expres

...

generalExpr =  forall_{A | A = FALSE} [not](A)
generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  (([not](TRUE) = TRUE) = FALSE) in BOOLEANS
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: [not](TRUE) = TRUE, y: FALSE}
AFTER _specialized_expr:
    specializedExpr =  (([not](TRUE) = TRUE) = FALSE) in BOOLEANS
    requirements =  []
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: [not](TRUE) = TRUE, y: FALSE}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (([not](TRUE) = TRUE) = FALSE) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
[_specialized_expression]subbed_expr =  [not]([not](TRUE) = TRUE)
[_specialized_expression]subbedConditions =  [([not](TRUE) = TRUE) = FALSE]
[_specialized_express

generalExpr =  forall_{A | [not](A) in BOOLEANS} (A in BOOLEANS)
[_specialized_expression]subbed_expr =  TRUE in BOOLEANS
[_specialized_expression]subbedConditions =  [[not](TRUE) in BOOLEANS]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), []]
[_specialized_expression]mappings =  {A: TRUE}
AFTER _specialized_expr:
    specializedExpr =  TRUE in BOOLEANS
    requirements =  [[not](TRUE) in BOOLEANS]
    mappedVarLists =  [(A,), []]
    mappings =  {A: TRUE}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  TRUE in BOOLEANS
generalTruth =  |- forall_{A | [not](A) in BOOLEANS} (A in BOOLEANS)
requirementTruths =  [|- [not](TRUE) in BOOLEANS]
generalExpr =  forall_{A in BOOLEANS} (A or [not](A))
[_specialized_expression]subbed_expr =  [not](TRUE) or [not]([not](TRUE))
[_specialized_expression]subbedConditions =  [[not](TRUE) in BOOLEANS]
[_specialized_expression]requirements =  []
[_spe

assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  ({TRUE , FALSE} = BOOLEANS) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  (BOOLEANS = {TRUE , FALSE}) in BOOLEANS
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: BOOLEANS, y: {TRUE , FALSE}}
AFTER _specialized_expr:
    specializedExpr =  (BOOLEANS = {TRUE , FALSE}) in BOOLEANS
    requirements =  []
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: BOOLEANS, y: {TRUE , FALSE}}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (BOOLEANS = {TRUE , FALSE}) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
[_specialized_expression

generalExpr =  forall_{A, B | A supseteq B} (B subseteq A)
[_specialized_expression]subbed_expr =  NaturalsPos subseteq Reals
[_specialized_expression]subbedConditions =  [Reals supseteq NaturalsPos]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), (B,), []]
[_specialized_expression]mappings =  {A: Reals, B: NaturalsPos}
AFTER _specialized_expr:
    specializedExpr =  NaturalsPos subseteq Reals
    requirements =  [Reals supseteq NaturalsPos]
    mappedVarLists =  [(A,), (B,), []]
    mappings =  {A: Reals, B: NaturalsPos}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  NaturalsPos subseteq Reals
generalTruth =  |- forall_{A, B | A supseteq B} (B subseteq A)
requirementTruths =  [|- Reals supseteq NaturalsPos]
generalExpr =  forall_{A, B | A subset B} (B supset A)
[_specialized_expression]subbed_expr =  Reals supset NaturalsPos
[_specialized_expression]subbedConditions =  [NaturalsP

generalExpr =  forall_{x, y | x < y} (y > x)
[_specialized_expression]subbed_expr =  1 > 0
[_specialized_expression]subbedConditions =  [0 < 1]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: 0, y: 1}
AFTER _specialized_expr:
    specializedExpr =  1 > 0
    requirements =  [0 < 1]
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: 0, y: 1}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  1 > 0
generalTruth =  |- forall_{x, y | x < y} (y > x)
requirementTruths =  [|- 0 < 1]
generalExpr =  forall_{x, y | x > y} (y < x)
[_specialized_expression]subbed_expr =  0 < 1
[_specialized_expression]subbedConditions =  [1 > 0]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: 1, y: 0}
AFTER _specialized_expr:
    specializedExpr =  0 < 1
    r

In [2]:
La = Literal('a')
Lb = Literal('b')
Lc = Literal('c')
Ld = Literal('d')
Le = Literal('e')

Le: e

In [3]:
basicForallExpr = Forall(x, Px, conditions=[Qx, Rx], domain=S)

basicForallExpr: forall_{x in S | Q(x) , R(x)} P(x)

In [4]:
Qa = Function(Q, a)
Ra = Function(R, a)
basicForallExpr.specialize({x:a}, assumptions=(basicForallExpr, InSet(a, S), Qa, Ra))

generalExpr =  forall_{x in S | Q(x) , R(x)} P(x)
[_specialized_expression]subbed_expr =  P(a)
[_specialized_expression]subbedConditions =  [a in S, Q(a), R(a)]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), []]
[_specialized_expression]mappings =  {x: a}
AFTER _specialized_expr:
    specializedExpr =  P(a)
    requirements =  [a in S, Q(a), R(a)]
    mappedVarLists =  [(x,), []]
    mappings =  {x: a}
assumptionsSet =  frozenset({forall_{x in S | Q(x) , R(x)} P(x)})
assumptions =  [forall_{x in S | Q(x) , R(x)} P(x), a in S, Q(a), R(a)]
specializedTruth assumptions =  (forall_{x in S | Q(x) , R(x)} P(x), a in S, Q(a), R(a))
specializedTruth expr =  P(a)
generalTruth =  {forall_{x in S | Q(x) , R(x)} P(x)} |- forall_{x in S | Q(x) , R(x)} P(x)
requirementTruths =  [{a in S} |- a in S, {Q(a)} |- Q(a), {R(a)} |- R(a)]


{forall_{x in S | Q(x) , R(x)} P(x) , a in S , Q(a) , R(a)} |- P(a)

In [5]:
Qa = Function(Q, a)
Ra = Function(R, a)
Rb = Function(R, b)
Lc = Literal('c')
Ld = Literal('d')
QLc = Function(Q, Lc)
RLc = Function(R, Lc)
basicForallExprSpec = basicForallExpr.specialize({x:Lc}, assumptions=(basicForallExpr, InSet(Lc, S), QLc, RLc))

generalExpr =  forall_{x in S | Q(x) , R(x)} P(x)
[_specialized_expression]subbed_expr =  P(c)
[_specialized_expression]subbedConditions =  [c in S, Q(c), R(c)]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), []]
[_specialized_expression]mappings =  {x: c}
AFTER _specialized_expr:
    specializedExpr =  P(c)
    requirements =  [c in S, Q(c), R(c)]
    mappedVarLists =  [(x,), []]
    mappings =  {x: c}
assumptionsSet =  frozenset({forall_{x in S | Q(x) , R(x)} P(x)})
assumptions =  [forall_{x in S | Q(x) , R(x)} P(x), c in S, Q(c), R(c)]
specializedTruth assumptions =  (forall_{x in S | Q(x) , R(x)} P(x), c in S, Q(c), R(c))
specializedTruth expr =  P(c)
generalTruth =  {forall_{x in S | Q(x) , R(x)} P(x)} |- forall_{x in S | Q(x) , R(x)} P(x)
requirementTruths =  [{c in S} |- c in S, {Q(c)} |- Q(c), {R(c)} |- R(c)]


basicForallExprSpec: {forall_{x in S | Q(x) , R(x)} P(x) , c in S , Q(c) , R(c)} |- P(c)

In [6]:
def testFxn():
    import pdb
    pdb.set_trace()
    return basicForallExpr.usedVars()

In [7]:
basicForallExprSpec.usedVars()

{P}

In [8]:
basicForallExprSpec.usedLiterals()

{c}

In [9]:
(list(basicForallExprSpec.subExprIter()))

[P, c]

In [10]:
list(((list(basicForallExprSpec.subExprIter()))[1]).subExprIter())

[]

In [11]:
Lc.markAsConstrained()

In [12]:
Lc._constrained

True

In [13]:
# notice now that re-marking as constrained does not elicit an error
Lc.markAsConstrained()

In [14]:
# Check if Lc is constrained. Returns False if no _constrained attribute;
# otherwise returns value of the _constrained attribute
Lc.isConstrained()

True

In [15]:
# but we can still delete the _constrained attribute, which is a little weird:
if hasattr(Lc, '_constrained'):
    delattr(Lc, '_constrained')
hasattr(Lc, '_constrained')

False

In [16]:
# now calling isConstrained will return false
# even though the attribute doesn't exist:
Lc.isConstrained()

False

In [17]:
basicThereExistsExpr00 = Exists(x, Px)

basicThereExistsExpr00: exists_{x} P(x)

In [18]:
basicThereExistsExpr00KT = basicThereExistsExpr00.prove(assumptions={basicThereExistsExpr00})

basicThereExistsExpr00KT: {exists_{x} P(x)} |- exists_{x} P(x)

In [19]:
basicThereExistsExpr01 = Exists(x, Px, conditions=[Qx], domain=S)

basicThereExistsExpr01: exists_{x in S | Q(x)} P(x)

In [20]:
basicThereExistsExpr01KT = basicThereExistsExpr01.prove(assumptions={basicThereExistsExpr01})

basicThereExistsExpr01KT: {exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [21]:
basicThereExistsExpr02 = Exists([x, y], Pxy, conditions=[Qx, Ry], domain=S)

basicThereExistsExpr02: exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [22]:
basicThereExistsExpr02

exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [23]:
basicThereExistsExpr02KT = basicThereExistsExpr02.prove(assumptions = {basicThereExistsExpr02} )

basicThereExistsExpr02KT: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [24]:
Txy = Function(T, {x, y})
basicThereExistsExpr03 = Exists([x, y], Pxy, conditions=[Txy, Qx, Ry], domain=S)

basicThereExistsExpr03: exists_{x, y in S | T(y , x) , Q(x) , R(y)} P(x , y)

In [25]:
basicThereExistsExpr03.instanceExpr.conditions

(y in S , T(y , x) , Q(x) , R(y))

In [26]:
(basicThereExistsExpr03.instanceExpr.allConditions()[1]).substituted({z:a})

T(y , x)

In [27]:
Tz = Function(T, z)
Pxyz = Function(P, [x, y, z])
basicThereExistsExpr04 = Exists([x, y, z], Pxyz, conditions=[Qx, Ry, Tz], domain=S)

basicThereExistsExpr04: exists_{x, y, z in S | Q(x) , R(y) , T(z)} P(x , y , z)

In [28]:
Tz = Function(T, z)
Pxyz = Function(P, [x, y, z])
Qxyz = Function(Q, [x, y, z])
basicThereExistsExpr05 = Exists([x, y, z], Pxyz, conditions=[Qxyz], domain=S)

basicThereExistsExpr05: exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)

In [93]:
basicThereExistsExpr05KT = basicThereExistsExpr05.prove(assumptions = {basicThereExistsExpr05} )

basicThereExistsExpr05KT: {exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)} |- exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)

In [29]:
Pxyzt = Function(P, [x, y, z, t])
Qxyzt = Function(Q, [x, y, z, t])
basicThereExistsExpr06 = Exists([x, y, z, t], Pxyzt, conditions=[Qxyzt], domain=S)

basicThereExistsExpr06: exists_{x, y, z, t in S | Q(x , y , z , t)} P(x , y , z , t)

In [30]:
basicThereExistsExpr00KT

{exists_{x} P(x)} |- exists_{x} P(x)

In [31]:
basicThereExistsExpr00KT

{exists_{x} P(x)} |- exists_{x} P(x)

In [32]:
QLc = Function(Q, Lc)
basicThereExistsExpr00KTSkolemized = basicThereExistsExpr00KT.skolemize({x:Lc}, assumptions={QLc, InSet(Lc, S)})

ENTERING KT.skolemize()
        processedSubMap so far =  {x: c}
ENTERING KT._skolemized_expressions


basicThereExistsExpr00KTSkolemized: {exists_{x} P(x)} |- P(c)

In [33]:
basicThereExistsExpr00KTSkolemized.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x} P(x)} |- P(c)
1	assumption		{exists_{x} P(x)} |- exists_{x} P(x)

In [34]:
basicThereExistsExpr01KT

{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [35]:
Ld = Literal('d')
QLd = Function(Q, Ld)
# A, B, C = (
#     basicThereExistsExpr01KT.skolemize(
#         {x:Ld},
#         assumptions={InSet(Ld, S), QLd}
#     )
# )
basicThereExistsExpr01KTSkolemized = (
    basicThereExistsExpr01KT.skolemize(
        {x:Ld},
        assumptions={InSet(Ld, S), QLd}
    )
)
# skolemizing just one of two possible instance vars
# This is not quite working — somehow subbing into one of the conditions
# but not explicitly then including that condition in the
# subbedConditions list
# basicThereExistsExpr03ThmSkolemized = (
#     basicThereExistsExpr03Thm.skolemize(
#         {y:Ld},
#         assumptions={InSet(Ld, S), RLd}
#     )
# )
A, B, C = basicThereExistsExpr01KTSkolemized

ENTERING KT.skolemize()
        processedSubMap so far =  {x: d}
ENTERING KT._skolemized_expressions


A: {exists_{x in S | Q(x)} P(x)} |- P(d)
B: {exists_{x in S | Q(x)} P(x)} |- d in S
C: {exists_{x in S | Q(x)} P(x)} |- Q(d)

In [36]:
A.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x in S | Q(x)} P(x)} |- P(d)
1	assumption		{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [37]:
B.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x in S | Q(x)} P(x)} |- d in S
1	assumption		{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [38]:
C.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x in S | Q(x)} P(x)} |- Q(d)
1	assumption		{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [39]:
basicThereExistsExpr02KT

{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [40]:
# intended to operate on expression, not full known truth
def skolemize2(inputExpr, skMap, nonSkVars=set()):
    from proveit.logic import Exists
    setOfResults = [inputExpr]
    print("setOfResults = ", setOfResults)
    print("len(setOfResults) = ", len(setOfResults))
    
    while len(skMap)>0:
        tempResults = []
        # currentSkMapping = {}
        for theExpr in setOfResults:
    
            # check if outer expression is an Exists operation
            if isinstance(theExpr, Exists):
                print("1 Input was an Exists operation.")
                print("    1.1 the input was theExpr = ", theExpr)
                tempInstanceVar = theExpr.instanceVar
                print("    1.2 with tempInstanceVar = ", tempInstanceVar)
                
                # check if skolemization appropriate
                # i.e. if instance variable appears in skMap as a key
                if tempInstanceVar in skMap:
                    print("    1.3 Found the instanceVar as a key in the map!")
                    # perform skolemization
                    tempInstanceExpr = theExpr.instanceExpr
                    print("        theExpr.tempInstanceExpr = ", tempInstanceExpr)
                    tempInstanceExprSubbed = tempInstanceExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]})
                    print("        tempInstanceExprSubbed = ", tempInstanceExprSubbed)
                    tempResults.append(tempInstanceExprSubbed)
                    print("        tempResults = ", tempResults)
                    tempConditions = theExpr.conditions
                    print("        tempConditions = ", tempConditions)
                    tempConditionsSubbed = []
                    tempConditionsCompletelySubbed = []
                    tempConditionsIncompletelySubbed = []
                    for oldExpr in tempConditions:
                        # interestingly here: the subbed conditions will not necessarily reduce to 
                        # completely skolemized conditions, and thus must be checked
                        # if completely skolemized, then continue as originally planned, appending
                        # them to the tempResults;
                        # if not completely skolemized, hold them separately in another list
                        # which will eventually need to be used in the “else” portion below
                        # can eventually delete the next line
                        newExpr = oldExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]})
                        # check if vars in newExpr include any vars in nonSkVars
                        print("        vars appearing in newExpr = ", newExpr.freeVars())
                        print("        vars appearing in nonSkVars = ", nonSkVars)
                        print("        vars in intersection: ", nonSkVars.intersection(newExpr.freeVars()))
                        tempIntersection = nonSkVars.intersection(newExpr.freeVars())
                        if len(tempIntersection)==0:
                            tempConditionsCompletelySubbed.append(newExpr)
                        else:
                            tempConditionsIncompletelySubbed.append(newExpr)
                            
                        tempConditionsSubbed.append(oldExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]}))
                        # the following used to produce flat list
                        tempResults.append(oldExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]}))
                        
                    print("        tempConditionsCompletelySubbed = ", tempConditionsCompletelySubbed)
                    print("        tempConditionsIncompletelySubbed = ", tempConditionsIncompletelySubbed)
                    # we make a list of lists, consisting of the
                    # instanceExpressionSubbed, tempConditionsCompletelySubbed, tempConditionsIncompletelySubbed
                    # tempResults.append(tempConditionsCompletelySubbed)
                    # tempResults.append(tempConditionsIncompletelySubbed)
                    print("        tempResults = ", tempResults)
                    # remove the Skolem constant from the skMap
                    del skMap[tempInstanceVar]
                    print("        skMap now = ", skMap)
                    # setOfResults = tempResults
                    print("        tempResults now = ", tempResults)
                    
                else:
                    print("    1.4 Did NOT find the instanceVar as a key in the map!")
                    nonSkVars.add(tempInstanceVar) # adding to a SET
                    # try constructing the input expression around an attempted
                    # skolemization on an inner expression (if it exists)
                    tempInstanceExpr = theExpr.instanceExpr
                    print("        tempInstanceExpr = ", tempInstanceExpr)
                    # for testing; delete the following line later
                    # print("    1.4 skolomize2() = ", skolemize2(tempInstanceExpr, skMap))
                    tempSkolemizeResult = skolemize2(tempInstanceExpr, skMap, nonSkVars)
                    print("    1.4 skolomize2() = ", tempSkolemizeResult)
                    A = tempSkolemizeResult[0]  # the 1st item
                    B = tempSkolemizeResult[1:] # all other items
                    print("        A = ", A)
                    print("        B = ", B)
                    print("        just after A, B, tempResults = ", tempResults)
                    tempConstructed = Exists(theExpr.instanceVar,
                                             A,
                                             conditions=theExpr.conditions)
                    print("        tempConstructed = ", tempConstructed)
                    tempResults.append(tempConstructed)
                    print("        tempResults = ", tempResults)
                    tempResults.extend(B)
                    print("        tempResults = ", tempResults)
                    

            else:
                print("2 Input was NOT an Exists operation.")
                print("    2.1 input was theExpr = ", theExpr)
                tempResults.append(theExpr) # perhaps we need to acknowledge a possible substitution?
                print("    2.2 tempResults = ", tempResults)
        
        # end for loop

        setOfResults = tempResults
    
    # end while loop
    
    print("Returning from skolomize2 call, setOfResults = ", setOfResults)
    return setOfResults
    
    
    

In [41]:
def skolemize3(inputExpr, skMap):
    '''
    inputExpr should be an Exists or nested Exists expressions;
    skMap is a dictionary of the form {x:La, y:Lb, ...} specifying
    the desired mapping(s) from Exists instance variable(s) to
    constant Literal(s).
    '''
    
    # from proveit.logic import Exists
    setOfResults = [inputExpr]

    while len(skMap)>0:
        tempResults = skolemizeHelper(setOfResults, skMap)
        setOfResults = [tempResults[0], *tempResults[1]]
        skMap = tempResults[3]
    # end while loop

    return setOfResults

In [42]:
def skolemizeHelper(setOfResults, skMap, nonSkVars=set()):
    
    from proveit.logic import Exists
    
    tempResults = []
    tempConditionsCompletelySubbed = []
    tempConditionsIncompletelySubbed = []
    
    for theExpr in setOfResults:

        # check if outer expression is an Exists operation
        if isinstance(theExpr, Exists):
            tempInstanceVar = theExpr.instanceVar

            # check if skolemization appropriate
            # i.e. if instance variable appears in skMap as a key
            if tempInstanceVar in skMap:
                # perform skolemization
                tempInstanceExpr = theExpr.instanceExpr
                tempInstanceExprSubbed = tempInstanceExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]})
                tempResults.append(tempInstanceExprSubbed)
                tempConditions = theExpr.conditions
                tempConditionsSubbed = []
                for oldExpr in tempConditions:
                    # Notice that any resulting subbed conditions will not necessarily reduce to 
                    # completely Skolemized conditions, and thus must be checked.
                    # If completely skolemized, append them to the completely subbed conditions;
                    # if not completely skolemized, hold them in the separate incompletely
                    # subbed conditions list which will eventually need to be used in the “else”
                    # portion below
                    newExpr = oldExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]})
                    # check if vars in newExpr include any vars in nonSkVars
                    tempIntersection = nonSkVars.intersection(newExpr.freeVars())
                    if len(tempIntersection)==0:
                        tempConditionsCompletelySubbed.append(newExpr)
                    else:
                        tempConditionsIncompletelySubbed.append(newExpr)

                    # probably don't need following line anymore
                    # tempConditionsSubbed.append(oldExpr.substituted({tempInstanceVar:skMap[tempInstanceVar]}))

                # remove the Skolem constant from the skMap
                del skMap[tempInstanceVar]

            else:
                nonSkVars.add(tempInstanceVar) # adding to a SET
                # try constructing the input expression around an attempted
                # skolemization on an inner expression (if it exists)
                tempInstanceExpr = theExpr.instanceExpr
                tempSkolemizeResult = skolemizeHelper([tempInstanceExpr], skMap, nonSkVars)
                A = tempSkolemizeResult[0]  # the 1st item
                B = tempSkolemizeResult[1]  # completely subbed conditions
                C = tempSkolemizeResult[2]  # incompletely subbed conditions
                if len(C) != 0:
                    theExprConditions = ExprList(*theExpr.conditions, *C)
                else:
                    theExprConditions = theExpr.conditions
                tempConstructed = Exists(theExpr.instanceVar,
                                         A,
                                         conditions=theExprConditions)
                tempResults.append(tempConstructed)
                # tempResults.append(B)
                tempConditionsCompletelySubbed.append(B)


        else:
            # tempResults.append(theExpr) # perhaps we need to acknowledge a possible substitution?
            # instead, pop them back into the tempConditionsCompletelySubbed set
            tempConditionsCompletelySubbed.append(theExpr)

    # end for loop
    
    # we compile the results, consisting of the
    # instanceExpressionSubbed, tempConditionsCompletelySubbed,
    # tempConditionsIncompletelySubbed, and the (possibly modified) skMap
    tempResults.append(tempConditionsCompletelySubbed)
    tempResults.append(tempConditionsIncompletelySubbed)
    tempResults.append(skMap)

    return tempResults

In [43]:
innerExists = basicThereExistsExpr02KT.instanceExpr

innerExists: exists_{y in S | R(y)} P(x , y)

In [44]:
innerExists.conditions

(y in S , R(y))

In [45]:
innerExists.instanceExpr.substituted({y:a},)

P(x , a)

In [46]:
innerExists.substituted({y:a},)

exists_{y in S | R(y)} P(x , y)

In [47]:
rebuiltExists = Exists((a,), innerExists.instanceExpr.substituted({y:a},), conditions=[innerExists.conditions.substituted({y:a},)])

rebuiltExists: exists_{a | (a in S , R(a))} P(x , a)

In [48]:
innerExists.conditions[0].freeVars()

{S, y}

In [49]:
basicThereExistsExpr02KT.expr

exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [50]:
tempResult0201 = skolemize3(basicThereExistsExpr02KT.expr, {x:a, y:b})

tempResult0201: (P(a , b) , b in S , R(b) , a in S , Q(a))

In [51]:
tempResult0201 = skolemize3(basicThereExistsExpr02KT.expr, {x:a})

tempResult0201: (exists_{y in S | R(y)} P(a , y) , a in S , Q(a))

In [52]:
tempResult0201 = skolemize3(basicThereExistsExpr02KT.expr, {y:b})

tempResult0201: (exists_{x in S | Q(x)} P(x , b) , (b in S , R(b)))

In [53]:
basicThereExistsExpr04

exists_{x, y, z in S | Q(x) , R(y) , T(z)} P(x , y , z)

In [54]:
tempResult0401 = skolemize3(basicThereExistsExpr04, {y:b, z:c})

tempResult0401: (exists_{x in S | Q(x)} P(x , b , c) , (c in S , T(c)) , (b in S , R(b)))

In [55]:
tempResult0402 = skolemize3(basicThereExistsExpr04, {x:a, y:b, z:c})

tempResult0402: (P(a , b , c) , c in S , T(c) , b in S , R(b) , a in S , Q(a))

In [56]:
tempResult0403 = skolemize3(basicThereExistsExpr04, {x:a, z:c})

tempResult0403: (exists_{y in S | R(y)} P(a , y , c) , (c in S , T(c)) , a in S , Q(a))

In [57]:
basicThereExistsExpr05

exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)

In [58]:
tempResult0501 = skolemize3(basicThereExistsExpr05, {x:La, y:Lb, z:Lc})

tempResult0501: (P(a , b , c) , c in S , Q(a , b , c) , b in S , a in S)

In [59]:
tempResult0502 = skolemize3(basicThereExistsExpr05, {y:Lb})

tempResult0502: (exists_{x in S} [exists_{z in S | Q(x , b , z)} P(x , b , z)] , (b in S))

In [60]:
tempResult0503 = skolemize3(basicThereExistsExpr05, {x:La, z:Lc})

tempResult0503: (exists_{y in S | Q(a , y , c)} P(a , y , c) , (c in S) , a in S)

In [61]:
basicThereExistsExpr06

exists_{x, y, z, t in S | Q(x , y , z , t)} P(x , y , z , t)

In [98]:
basicThereExistsExpr06

exists_{x, y, z, t in S | Q(x , y , z , t)} P(x , y , z , t)

In [62]:
tempResult0601 = skolemize3(basicThereExistsExpr06, {x:La, y:Lb, z:Lc, t:Ld})

tempResult0601: (P(a , b , c , d) , d in S , Q(a , b , c , d) , c in S , b in S , a in S)

In [63]:
tempResult0602 = skolemize3(basicThereExistsExpr06, {y:Lb, t:Ld})

tempResult0602: (exists_{x in S} [exists_{z in S | Q(x , b , z , d)} P(x , b , z , d)] , ((d in S)) , (b in S))

In [64]:
tempResult0603 = skolemize3(basicThereExistsExpr06, {t:Ld})

tempResult0603: (exists_{x in S} [exists_{y in S} [exists_{z in S | Q(x , y , z , d)} P(x , y , z , d)]] , (((d in S))))

In [65]:
basicThereExistsExpr02KT

{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [66]:
La = Literal('a')              # 'a'
Lb = Literal('b')              # 'b'
Qa = Function(Q, La)           # Q(a)
Rb = Function(R, Lb)           # R(b)
Pab = Function(P, {La, Lb})    # P(a, b)
Ry = Function(R, y)            # R(y)
# basicThereExistsExpr02KTSkolemized = (
#     basicThereExistsExpr02KT.skolemize(
#         {x:La}, assumptions={InSet(La, S), Qa})
# )
basicThereExistsExpr02KTSkolemized = (
    basicThereExistsExpr02KT.skolemize(
        {x:La, y:Lb}, assumptions={})
)
# basicThereExistsExpr02KTSkolemized = (
#     basicThereExistsExpr02KT.skolemize(
#         {x:La, y:Lb}, assumptions={})
# )
# basicThereExistsExpr02KTSkolemized
basicThereExistsExpr02KTSkolemized
# D, E, F, G, H = basicThereExistsExpr02KTSkolemized

ENTERING KT.skolemize()
        processedSubMap so far =  {x: a}
        processedSubMap so far =  {x: a, y: b}
ENTERING KT._skolemized_expressions


[{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- P(a , b),
 {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- a in S,
 {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- Q(a),
 {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- b in S,
 {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- R(b)]

In [67]:
len(basicThereExistsExpr02KTSkolemized)

5

In [68]:
D, E, F, G, H = basicThereExistsExpr02KTSkolemized

D: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- P(a , b)
E: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- a in S
F: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- Q(a)
G: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- b in S
H: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- R(b)

In [69]:
# D, E, F = basicThereExistsExpr02KTSkolemized

In [70]:
D.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- P(a , b)
1	assumption		{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [71]:
E.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- a in S
1	assumption		{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [72]:
F.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- Q(a)
1	assumption		{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [96]:
basicThereExistsExpr05KT

{exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)} |- exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)

In [97]:
La = Literal('a')              # 'a'
Lb = Literal('b')              # 'b'
Lc = Literal('c')              # 'c'
Qa = Function(Q, La)           # Q(a)
Rb = Function(R, Lb)           # R(b)
Pab = Function(P, {La, Lb})    # P(a, b)
Ry = Function(R, y)            # R(y)

basicThereExistsExpr05KTSkolemized = (
    basicThereExistsExpr05KT.skolemize(
        {x:La, y:Lb}, assumptions={})
)
basicThereExistsExpr05KTSkolemized

ENTERING KT.skolemize()
        processedSubMap so far =  {x: a}
        processedSubMap so far =  {x: a, y: b}
ENTERING KT._skolemized_expressions


[{exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)} |- exists_{z in S | Q(a , b , z)} P(a , b , z),
 {exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)} |- a in S,
 {exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)} |- b in S]

In [73]:
# QLc = Function(Q, Lc)
# RLd = Function(R, Ld)
# PLcLd = Function(P, {Lc, Ld})
# # the following should be problematic if it attempts to re-use the x:Lc skolemization mapping
# # because the Lc Literal was previously used in a skolemization above:
# # basicThereExistsExpr03Thm.skolemize({x:Lc, y:Ld}, assumptions={InSet(Lc, S), InSet(Ld, S), QLc, RLd, PLcLd})


# from proveit import ProofFailure
# try:
#     basicThereExistsExpr03Thm.skolemize({x:Lc, y:Ld}, assumptions={InSet(Lc, S), InSet(Ld, S), QLc, RLd, PLcLd})
#     assert False, "Expecting a Skolemization error; should not make it to this point."
# except ProofFailure as e:
#     print("EXPECTED ERROR:", e)

In [74]:
# Le = Literal('e')
# QLe = Function(Q, Le)
# RLd = Function(R, Ld)
# PLeLd = Function(P, {Le, Ld})
# # the following should NOT be problematic since we map x and y to previously unused Literals Le and Ld
# # for the skolemization mapping:
# basicThereExistsExpr01KTSkolemized = (
#     basicThereExistsExpr01KT.skolemize(
#         {x:Le, y:Ld},
#         assumptions={InSet(Le, S), InSet(Ld, S), QLe, RLd, PLeLd}
#     )
# )
# skolemizing just one of two possible instance vars
# This is not quite working — somehow subbing into one of the conditions
# but not explicitly then including that condition in the
# subbedConditions list
# basicThereExistsExpr03ThmSkolemized = (
#     basicThereExistsExpr03Thm.skolemize(
#         {y:Ld},
#         assumptions={InSet(Ld, S), RLd}
#     )
# )

Attempting to skolemize a Forall expression should fail and produce an error message, alerting the user to the fact that only an Exists expression can be skolemized:

In [75]:
from proveit import ProofFailure
try:
    substitution.skolemize({x:Lc}, assumptions={Qa})
    assert False, "Expecting a Skolemization error; should not make it to this point."
except ProofFailure as e:
    print("EXPECTED ERROR:", e)

ENTERING KT.skolemize()
        processedSubMap so far =  {x: c}
EXPECTED ERROR: Proof step failed assuming {Q(a)}: May only skolemize instance variables of an Exists expression.


But right now, we can still specialize a Forall $\forall$ KnownTruth using a Literal that has been previously used as a Skolem constant:

In [76]:
substitutionSpec = substitution.specialize({x:Lc, y:Ld}, assumptions=[Equals(Lc, Ld)])

generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  (c = d) in BOOLEANS
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: c, y: d}
AFTER _specialized_expr:
    specializedExpr =  (c = d) in BOOLEANS
    requirements =  []
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: c, y: d}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (c = d) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
generalExpr =  forall_{A in BOOLEANS} (A or [not](A))
[_specialized_expression]subbed_expr =  (c = d) or [not](c = d)
[_specialized_expression]subbedConditions =  [(c = d) in BOOLEANS]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), []]
[_specialized_expression]mappings =  {

    requirements =  []
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: f(d), y: f(c)}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (f(d) = f(c)) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
generalExpr =  forall_{A in BOOLEANS} (A or [not](A))
[_specialized_expression]subbed_expr =  (f(d) = f(c)) or [not](f(d) = f(c))
[_specialized_expression]subbedConditions =  [(f(d) = f(c)) in BOOLEANS]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), []]
[_specialized_expression]mappings =  {A: f(d) = f(c)}
AFTER _specialized_expr:
    specializedExpr =  (f(d) = f(c)) or [not](f(d) = f(c))
    requirements =  [(f(d) = f(c)) in BOOLEANS]
    mappedVarLists =  [(A,), []]
    mappings =  {A: f(d) = f(c)}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (f(d) = f(c)) or [not](f(d) =

substitutionSpec: {c = d} |- f(c) = f(d)

## Working to understand more carefully what happens in the instantiation of a simple Forall expression using ``specialize()``, so we can better understand how to implement the skolemize.

In [77]:
basicForAllExpr = Forall(x, Px)

basicForAllExpr: forall_{x} P(x)

In [78]:
basicForAllExpr

forall_{x} P(x)

In [79]:
# from proveit._core_ import KnownTruth
# tempKT = KnownTruth(basicForAllExpr, {basicForAllExpr})

In [80]:
# print(tempKT)

In [81]:
theOperand = basicForAllExpr.operand

theOperand: x -> P(x)

In [82]:
theOperand.parameterVars

(x,)

In [83]:
theOperand.body

P(x)

In [84]:
theOperand.conditions

()

In [85]:
basicForAllExprThm = basicForAllExpr.prove(assumptions = {basicForAllExpr})

basicForAllExprThm: {forall_{x} P(x)} |- forall_{x} P(x)

In [86]:
basicForAllExprThm.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 2,
1,Literal,,
2,Lambda,parameter: 5body: 3,
3,Operation,operator: 4operand: 5,
4,Variable,,
5,Variable,,


In [87]:
basicForAllExprThm.specialize({x:Lc})

generalExpr =  forall_{x} P(x)
[_specialized_expression]subbed_expr =  P(c)
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), []]
[_specialized_expression]mappings =  {x: c}
AFTER _specialized_expr:
    specializedExpr =  P(c)
    requirements =  []
    mappedVarLists =  [(x,), []]
    mappings =  {x: c}
assumptionsSet =  frozenset({forall_{x} P(x)})
assumptions =  [forall_{x} P(x)]
specializedTruth assumptions =  (forall_{x} P(x),)
specializedTruth expr =  P(c)
generalTruth =  {forall_{x} P(x)} |- forall_{x} P(x)
requirementTruths =  []


{forall_{x} P(x)} |- P(c)

In [88]:
basicForAllExprThm.specialize({x:Lc}).proof()

generalExpr =  forall_{x} P(x)
[_specialized_expression]subbed_expr =  P(c)
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), []]
[_specialized_expression]mappings =  {x: c}
AFTER _specialized_expr:
    specializedExpr =  P(c)
    requirements =  []
    mappedVarLists =  [(x,), []]
    mappings =  {x: c}
assumptionsSet =  frozenset({forall_{x} P(x)})
assumptions =  [forall_{x} P(x)]
specializedTruth assumptions =  (forall_{x} P(x),)
specializedTruth expr =  P(c)
generalTruth =  {forall_{x} P(x)} |- forall_{x} P(x)
requirementTruths =  []


step type	requirements	statement
0	specialization	1	{forall_{x} P(x)} |- P(c)
	x : c
1	assumption		{forall_{x} P(x)} |- forall_{x} P(x)

In [89]:
basicForAllExpr02 = Forall(x, Px, conditions={Qx})

basicForAllExpr02: forall_{x | Q(x)} P(x)

In [90]:
basicForAllExpr02Thm = basicForAllExpr02.prove(assumptions={basicForAllExpr02})

basicForAllExpr02Thm: {forall_{x | Q(x)} P(x)} |- forall_{x | Q(x)} P(x)

In [91]:
basicForAllExpr02Thm.specialize({x:Lc}, assumptions={QLc})

generalExpr =  forall_{x | Q(x)} P(x)
[_specialized_expression]subbed_expr =  P(c)
[_specialized_expression]subbedConditions =  [Q(c)]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), []]
[_specialized_expression]mappings =  {x: c}
AFTER _specialized_expr:
    specializedExpr =  P(c)
    requirements =  [Q(c)]
    mappedVarLists =  [(x,), []]
    mappings =  {x: c}
assumptionsSet =  frozenset({forall_{x | Q(x)} P(x)})
assumptions =  [Q(c), forall_{x | Q(x)} P(x)]
specializedTruth assumptions =  (Q(c), forall_{x | Q(x)} P(x))
specializedTruth expr =  P(c)
generalTruth =  {forall_{x | Q(x)} P(x)} |- forall_{x | Q(x)} P(x)
requirementTruths =  [{Q(c)} |- Q(c)]


{Q(c) , forall_{x | Q(x)} P(x)} |- P(c)

In [92]:
%end skolemization_explorations